In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('Test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 12:51:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path = 'airtrafficdata/airtraffic_all/airtraffic-part/flightmonth=200801/part-00252-5cde1303-4ebf-4a12-8fad-f5d9f9c9124a-Copy1.c000.snappy.parquet'

In [3]:
airtraffic = spark.read.parquet(path)

In [4]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [5]:
# Get daily count of cancelled flights where data is sorted in ascending order by count of cancelled flights.

In [6]:
airtraffic.filter(col('Cancelled')==1). \
    groupby(concat(col('Year'),lpad(col('Month'),2,'0'),lpad(col('DayofMonth'),2,'0')).alias('Date')). \
    agg(count(lit('1')).alias('Cancelled Count')).orderBy('Cancelled Count').show(31)

+--------+---------------+
|    Date|Cancelled Count|
+--------+---------------+
|20080112|            226|
|20080118|            230|
|20080120|            247|
|20080115|            299|
|20080124|            322|
|20080110|            341|
|20080113|            359|
|20080109|            377|
|20080126|            416|
|20080105|            456|
|20080108|            463|
|20080103|            475|
|20080121|            475|
|20080102|            511|
|20080111|            524|
|20080125|            526|
|20080123|            530|
|20080116|            532|
|20080101|            552|
|20080107|            579|
|20080127|            638|
|20080128|            654|
|20080106|            683|
|20080130|            694|
|20080104|            769|
|20080122|            788|
|20080117|            872|
|20080119|            876|
|20080129|            889|
|20080114|            909|
|20080131|           1081|
+--------+---------------+



In [7]:
airtraffic.filter(col('Cancelled')==1). \
    groupby(concat(col('Year'),lpad(col('Month'),2,'0'),lpad(col('DayofMonth'),2,'0')).alias('Date')). \
    agg(count(lit('1')).alias('Cancelled Count')).orderBy(col('Cancelled Count').desc()).show(31)

+--------+---------------+
|    Date|Cancelled Count|
+--------+---------------+
|20080131|           1081|
|20080114|            909|
|20080129|            889|
|20080119|            876|
|20080117|            872|
|20080122|            788|
|20080104|            769|
|20080130|            694|
|20080106|            683|
|20080128|            654|
|20080127|            638|
|20080107|            579|
|20080101|            552|
|20080116|            532|
|20080123|            530|
|20080125|            526|
|20080111|            524|
|20080102|            511|
|20080103|            475|
|20080121|            475|
|20080108|            463|
|20080105|            456|
|20080126|            416|
|20080109|            377|
|20080113|            359|
|20080110|            341|
|20080124|            322|
|20080115|            299|
|20080120|            247|
|20080118|            230|
|20080112|            226|
+--------+---------------+



In [8]:
#
# Project Year, Month, DayOfMonth, CRSDepTime and Origin for this task.
# Sort the data based up on year, month, day of month and then using scheduled time. 
# Data should be sorted in ascending order by year, month and day of month then in descending order by scheduled time.
#

In [9]:
airtraffic.select('Year','Month','DayOfMonth','CRSDepTime','Origin'). \
    orderBy('Year','Month','DayOfMonth',col('CRSDepTime').desc()). \
    show()

+----+-----+----------+----------+------+
|Year|Month|DayOfMonth|CRSDepTime|Origin|
+----+-----+----------+----------+------+
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   PHX|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   PHX|
|2008|    1|         1|      2359|   SLC|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   TUS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   SLC|
|2008|    1|         1|      2359|   SEA|
|2008|    1|         1|      2358|   LAS|
|2008|    1|         1|      2358|   LAS|
|2008|    1|         1|      2356|   LAS|
+----+-----+----------+----------+

In [14]:
# Create employees Data Frame and get employees data in ascending order by nationality.
# However, data related to United States should come at top always.

In [10]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 2,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [11]:
employeesDF = spark.createDataFrame(employees).toDF('id','FName','Lname','Salary','Bonus','Country','Phone','SSN')

In [13]:
employeesDF.show()

+---+-----+------+------+-----+--------------+----------------+-----------+
| id|FName| Lname|Salary|Bonus|       Country|           Phone|        SSN|
+---+-----+------+------+-----+--------------+----------------+-----------+
|  1|Scott| Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|  2|Henry|  Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|  3| Nick|Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|  4| Bill| Gomes|1500.0|    2|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+---+-----+------+------+-----+--------------+----------------+-----------+



In [16]:
employeesDF.orderBy('country').show()

+---+-----+------+------+-----+--------------+----------------+-----------+
| id|FName| Lname|Salary|Bonus|       Country|           Phone|        SSN|
+---+-----+------+------+-----+--------------+----------------+-----------+
|  4| Bill| Gomes|1500.0|    2|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|  2|Henry|  Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|  3| Nick|Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|  1|Scott| Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
+---+-----+------+------+-----+--------------+----------------+-----------+



In [19]:
employeesDF.withColumn('sort',when(upper(col('country')) == 'UNITED STATES','0').otherwise('1')).orderBy('sort','Country').show()

+---+-----+------+------+-----+--------------+----------------+-----------+----+
| id|FName| Lname|Salary|Bonus|       Country|           Phone|        SSN|sort|
+---+-----+------+------+-----+--------------+----------------+-----------+----+
|  1|Scott| Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|   0|
|  4| Bill| Gomes|1500.0|    2|     AUSTRALIA|+61 987 654 3210|789 12 6118|   1|
|  2|Henry|  Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|   1|
|  3| Nick|Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|   1|
+---+-----+------+------+-----+--------------+----------------+-----------+----+



In [20]:
# Sort the data in employeesDF using bonus. Data should be sorted numerically and null and empty values should come at the end.

In [42]:
employeesDF. \
    orderBy(col('Bonus').cast('int').asc_nulls_last()). \
    show()

+---+-----+------+------+-----+--------------+----------------+-----------+
| id|FName| Lname|Salary|Bonus|       Country|           Phone|        SSN|
+---+-----+------+------+-----+--------------+----------------+-----------+
|  4| Bill| Gomes|1500.0|    2|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|  1|Scott| Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|  3| Nick|Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|  2|Henry|  Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
+---+-----+------+------+-----+--------------+----------------+-----------+

